In [1]:
import numpy as np
import random
from numpy.random import normal
import sys
from random import randrange, shuffle

In [2]:
%cd Desktop/Technosphere/IR2/

/Users/an.novikova/Desktop/Technosphere/IR2


In [3]:
data_c1 = open("c1.txt", "r")
data_c2 = open("c2.txt", "r")

In [6]:
# data_c1.readline()

'1 1 1 0 0 0 0 0 0 0\n'

In [7]:
# data_c2.readline()

'0 0 0 0 0 0 0 0 0 0\n'

In [4]:
sessions_1 = []

for line in data_c1:
    l = [int(x) for x in line.strip().split()]
    sessions_1.append(np.asarray(l))

In [5]:
sessions_1[0]

array([1, 1, 1, 0, 0, 0, 0, 0, 0, 0])

In [6]:
sessions_2 = []

for line in data_c2:
    l = [int(x) for x in line.strip().split()]
    sessions_2.append(np.asarray(l))

In [7]:
sessions_in_group = 1000.0
cnt_group = len(sessions_1) / sessions_in_group

CTRs_1, CTRs_2 = [], []

AverClicks_1, AverClicks_2 = [], []

cnt_sessions = 0
cur_CTR_1, cur_CTR_2 = 0, 0
cur_AverClick_1, cur_AverClick_2 = 0, 0

In [8]:
for i in range(len(sessions_1)):
    if cnt_sessions == sessions_in_group:
        CTRs_1.append(cur_CTR_1 / sessions_in_group)
        CTRs_2.append(cur_CTR_2 / sessions_in_group)
        AverClicks_1.append(cur_AverClick_1 / sessions_in_group)
        AverClicks_2.append(cur_AverClick_2 / sessions_in_group)
        
        cur_CTR_1, cur_CTR_2 = 0, 0
        cur_AverClick_1, cur_AverClick_2 = 0, 0
        cnt_sessions = 0
    
    cur_CTR_1 += 1 if (sessions_1[i].sum() > 0) else 0
    cur_CTR_2 += 1 if (sessions_2[i].sum() > 0) else 0
    cur_AverClick_1 += sessions_1[i].sum() / float(len(sessions_1[i]))
    cur_AverClick_2 += sessions_2[i].sum() / float(len(sessions_2[i]))
    cnt_sessions += 1
    
if cnt_sessions == sessions_in_group:
    CTRs_1.append(cur_CTR_1 / sessions_in_group)
    CTRs_2.append(cur_CTR_2 / sessions_in_group)
    AverClicks_1.append(cur_AverClick_1 / sessions_in_group)
    AverClicks_2.append(cur_AverClick_2 / sessions_in_group)

In [9]:
CTRs_1 = np.asarray(CTRs_1)
CTRs_2 = np.asarray(CTRs_2)
AverClicks_1 = np.asarray(AverClicks_1)
AverClicks_2 = np.asarray(AverClicks_2)

In [10]:
print "CTRs_1.mean is", CTRs_1.mean()
print "CTRs_2.mean is", CTRs_2.mean()

CTRs_1.mean is 0.699946
CTRs_2.mean is 0.670108


In [11]:
print "AverClicks_1.mean is", AverClicks_1.mean()
print "AverClicks_2.mean is", AverClicks_2.mean()

AverClicks_1.mean is 0.1619103
AverClicks_2.mean is 0.0862814


In [12]:
# Процедура Bootstrap

def GetBootstrap(sessions):
    return np.random.choice(sessions, size=len(sessions), replace=True)

In [13]:
# Подсчет t-значения

_value = 1.96

def calc_t_criterion(split_1, split_2, t_importance = _value):
    t_value = abs(split_1.mean() - split_2.mean()) / \
        np.sqrt((split_1.var(ddof = 1) / float(split_1.shape[0])) + (split_2.var(ddof = 1) / float(split_2.shape[0])))
    return 1 if t_value > t_importance else 0

In [14]:
# Подсчет ASL

def calc_asl(split_1, split_2, bootstrap_iter):
    asl_metric = 0
    for it in range(bootstrap_iter):
        if it % 1000 == 0:
            sys.stderr.write('\rBootstrap iteration number is: ' + str(it))
        bs_split_1 = GetBootstrap(split_1)
        bs_split_2 = GetBootstrap(split_2)
        
        asl_metric += calc_t_criterion(bs_split_1, bs_split_2)
    
    return asl_metric / float(bootstrap_iter)

In [15]:
print "ASL metric for CTR is: ", calc_asl(CTRs_1, CTRs_2, bootstrap_iter=1000000)

ASL metric for CTR is: 

Bootstrap iteration number is: 999000

 1.0


In [17]:
print "ASL metric for AverCountOfClicks:", calc_asl(AverClicks_1, AverClicks_2, bootstrap_iter=1000000)

 ASL metric for AverCountOfClicks:

Bootstrap iteration number is: 999000

 1.0


#### Вывод: отвергаем гипотезу H0